In [3]:
import json
from bs4 import BeautifulSoup
import html
import re

In [4]:
b = json.load(open('barbara.json'))

In [5]:
m = json.load(open('michael.json'))

In [6]:
def munge_text(text):
    text = re.sub('<[^<]+?>', '', text)
    text = html.unescape(text)
    soup = BeautifulSoup(text)
    if soup.find('body'):
        soup = soup.find('body')
    text = "\n".join([c.prettify() for c in soup.contents if len(str(c)) > 1 and c.name != 'div'])
    text = text.replace("``", "\"")
    text = text.replace("''", "\"")
    return text

In [16]:
from difflib import SequenceMatcher

def similar(a, b):
    return SequenceMatcher(None, a.lower().strip(), b.lower().strip()).ratio()

def norm(s):
    return re.sub(r'[^\w]', '', s)

def similar_enough(a,b):
    if norm(a) in norm(b):
        return True
    elif norm(b) in norm(a):
        return True
    else:
        return similar(a,b) > 0.75

In [8]:
def first_graf(s):
    return s

In [17]:
from collections import defaultdict

def munge_pub(pub):
    return re.sub(r'\s*\([^\)]*\)\s*', '', pub).strip().replace('LAT', 'Los Angeles Times')

def handle_articles(articles, author):
    res = {}
    titles = {}
    dates = {}
    i = 0
    for a_ in articles:
        a = dict(a_)
        a['pub'] = munge_pub(a['pub'])
        if a['title'] in titles:
            id = titles[a['title']]
            existing = res[id]
            if similar_enough(existing['date'], a['date']) or similar_enough(first_graf(existing['text']), first_graf(a['text'])):
                res[id]['pub'] = list(set(res[id]['pub'] + [a['pub']]))
                continue
        if a['date'] in dates:
            id = dates[a['date']]
            existing = res[id]
            if similar_enough(existing['title'], a['title']) or similar_enough(first_graf(existing['text']), first_graf(a['text'])):
                res[id]['pub'] = list(set(res[id]['pub'] + [a['pub']]))
                continue
        a['id'] = i
        titles[a['title']] = i
        dates[a['date']] = i
        a['text'] = munge_text(a['text'])
        a['author'] = author
        a['pub'] = [a['pub']]
        res[i] = a
        i += 1

    return list(res.values())

In [18]:
b2 = handle_articles(b, 'Barbara Slavin')

In [19]:
m2 = handle_articles(m, 'Michael Ross')

In [20]:
!mkdir -p _data

In [21]:
with open('_data/michael.json', 'w') as f:
    f.write(json.dumps([mi for mi in sorted(m2, key=lambda mi: mi['date']) if mi['date'] > '1980-01-01'], indent=4))

In [22]:
with open('_data/barbara.json', 'w') as f:
    f.write(json.dumps(sorted(b2, key=lambda bi: bi['date']), indent=4))